# Gorilla assessment

In [189]:
import numpy as np
import pandas as pd

In [190]:
#date as index enables timeseries operations later on
forecast = pd.read_excel('data.xlsx', sheet_name='Forecasted Consumption')
meters = pd.read_excel('data.xlsx', sheet_name='Meter List')
rates = pd.read_excel('data.xlsx', sheet_name='Rates')

forecast.dtypes

Date        datetime64[ns]
Meter ID             int64
kWh                float64
dtype: object

In [191]:
forecast.head()

,Date,Meter ID,kWh
0,2020-10-01,1000000603,28.782474
1,2020-10-01,10588707,126.367711
2,2020-10-01,10626610,326.240595
3,2020-10-02,1000000603,26.690797
4,2020-10-02,10588707,118.322449


In [192]:
meters.head()

,Meter ID,Exit Zone,Annual Quantity (kWh)
0,1000000603,SC1,12337
1,10588707,EM2,75123
2,10626610,NW1,151779


In [193]:
rates.head()

,Date,Exit Zone,Annual Quantity (Min),Annual Quantity (Max),Rate (p/kWh)
0,2020-04-01,EA1,0,73200.0,0.0287
1,2020-04-01,EA2,0,73200.0,0.0287
2,2020-04-01,EA3,0,73200.0,0.0287
3,2020-04-01,EA4,0,73200.0,0.0287
4,2020-04-01,EM1,0,73200.0,0.0287


# Farid's Solution

We want to match the correct rate to the meter, so we inner join on exit Zone, we'd like to join on Annual Quantity as well but it's not an exact join. We'll fix this later.

In [194]:
result = meters.merge(rates, on="Exit Zone")

result.head()

,Meter ID,Exit Zone,Annual Quantity (kWh),Date,Annual Quantity (Min),Annual Quantity (Max),Rate (p/kWh)
0,1000000603,SC1,12337,2020-04-01,0,73200.0,0.0333
1,1000000603,SC1,12337,2020-04-01,73200,732000.0,0.0297
2,1000000603,SC1,12337,2020-04-01,732000,NaN,0.2431
3,1000000603,SC1,12337,2020-10-01,0,73200.0,0.0333
4,1000000603,SC1,12337,2020-10-01,73200,732000.0,0.0297


We have to filter the Annual Quantity between Max and Min to get the correct rate per meter, but there are empties in Max, fix this

In [195]:
result["Annual Quantity (Max)"] = result["Annual Quantity (Max)"].fillna(np.inf)

Preform the filter, throw away unnecessary columns

In [196]:
result = result[result["Annual Quantity (kWh)"].between(result["Annual Quantity (Min)"], result["Annual Quantity (Max)"])]
result = result[["Meter ID", "Date", "Rate (p/kWh)"]]

result.head()

,Meter ID,Date,Rate (p/kWh)
0,1000000603,2020-04-01,0.033300
3,1000000603,2020-10-01,0.033300
6,1000000603,2021-04-01,0.033313
9,1000000603,2021-10-01,0.033313
12,1000000603,2022-04-01,0.034706


Now we have to join the forecast on date, one possible solution is resampling the result first to a daily level.

In [197]:
result = result.set_index("Date").groupby("Meter ID", as_index=False).resample("D").agg("ffill")

result.head()

Meter ID  Rate (p/kWh)
  Date                              
0 2020-04-01  10588707        0.0228
  2020-04-02  10588707        0.0228
  2020-04-03  10588707        0.0228
  2020-04-04  10588707        0.0228
  2020-04-05  10588707        0.0228

In [198]:
result = result.merge(forecast, on=["Meter ID", "Date"])

result.head()

,Meter ID,Date,Rate (p/kWh),kWh
0,10588707,2020-10-01,0.0228,126.367711
1,10588707,2020-10-02,0.0228,118.322449
2,10588707,2020-10-03,0.0228,94.272125
3,10588707,2020-10-04,0.0228,97.641038
4,10588707,2020-10-05,0.0228,138.761245


In [199]:
result["Daily Cost"] = result["Rate (p/kWh)"] * result["kWh"] / 100

result.head()

,Meter ID,Date,Rate (p/kWh),kWh,Daily Cost
0,10588707,2020-10-01,0.0228,126.367711,0.028812
1,10588707,2020-10-02,0.0228,118.322449,0.026978
2,10588707,2020-10-03,0.0228,94.272125,0.021494
3,10588707,2020-10-04,0.0228,97.641038,0.022262
4,10588707,2020-10-05,0.0228,138.761245,0.031638


In [200]:
result = result.groupby("Meter ID").agg({"Daily Cost": "sum", "kWh": "sum"})

result = result.rename(columns={"Daily Cost": "Total Cost (£)", "kWh": "Total Estimated Consumption (kWh)"})

result = result.round(2)

In [201]:
result

,Total Cost (£),Total Estimated Consumption (kWh)
Meter ID,,
10588707,28.50,122450.0
10626610,88.24,303558.0
1000000603,8.27,24674.0


In [202]:
def get_qty(meter_ID):
    """
    extract annual forecasted quantity
    in: meter_ID (int)
    returns: qty (int)
    """
    #filter for meter
    meter = meters.loc[meter_ID]
    
    #sum full time period
    qty = meter['Annual Quantity (kWh)']
    
    return qty

def get_exit(meter_ID):
    """
    extract exit zone
    in: meter_ID (int)
    returns: exit (str)
    """
    #filter for meter
    meter = meters.loc[meter_ID]
    
    #sum full time period
    exit = meter['Exit Zone']
    
    return exit

In [ ]:
#test
# meter_ID = 1000000603
# print(get_exit(meter_ID))
# print(get_qty(meter_ID))

### Function to calculate day rate

In [ ]:
def get_rate_date(date): 
    """finds the latest applicable date for rate"""
    pre = rates.loc[rates.index < date]
    date = pre.index.max()
    return pd.to_datetime(date, format='%Y-%m-%d')

#test
#print(get_rate_date('20210202'))

def calc_rate(meter_ID, date): 
    """get rates for meter_ID on given date
    meter_ID (int)
    date (pd.dt object)
    return: rate (float)
    """
    exit = get_exit(meter_ID)
    qty = get_qty(meter_ID)
    rate_date = get_rate_date(date)
    
    #get rates: filter for exit zone, dates and forecasts
    rate = rates.get(rates['Exit Zone'] == exit)
    rate = rate.get(rate.index == rate_date)
    rate = rate.get(rate['Annual Quantity (Min)'] < qty)
    rate = rate.get(rate['Annual Quantity (Min)'] == rate['Annual Quantity (Min)'].max())
        
    return rate['Rate (p/kWh)'].iloc[0]

#test
#print(calc_rate(10626610,'20210202'))
    

### Add day rate to forecast

In [ ]:
#duplicate forecast
forecast_rate = forecast

#add col with day rate using list comprehension
forecast_rate['Day rate'] = [calc_rate(meter_ID,  date) for meter_ID, date in zip(forecast['Meter ID'], forecast.index)]

forecast_rate.head()


### Calculate daily cost

In [ ]:
forecast_rate['Day cost'] = forecast_rate['kWh'] * forecast_rate['Day rate']

forecast_rate.head()

### Calculate total cost

In [ ]:
#copy index from meters dataframe
total_consumption = pd.DataFrame(index=meters.index)

def sum_meter(meter_ID, col):
    """
    function intended for summing forecasted consumption or cost of passed meter
    meter_ID = (int)
    col = name of column to be summed (string)
    return: sum of filtered column (float)
    """
    filtered = forecast_rate.get(forecast_rate['Meter ID'] == meter_ID)
    
    return filtered[col].sum()
    
    
#calculate estimated total consumption by summing dayly forecast
total_consumption['Total Estimated Consumption (kWh)'] = [sum_meter(meter_ID, 'kWh') for meter_ID in total_consumption.index.values]

#calculate estimated total consumption by summing dayly forecast
total_consumption['Total Cost (£)'] = [sum_meter(meter_ID, 'Day cost') for meter_ID in total_consumption.index.values]
#convert pence to £
total_consumption['Total Cost (£)'] = total_consumption['Total Cost (£)']/100

total_consumption = total_consumption.round(2)
total_consumption.head()